In [1]:
import sys
sys.path.insert(0, r'../')
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import torch.optim as optim
import torch.nn as nn
import torch
import pickle
from sklearn.model_selection import train_test_split
import os
from utils.f1_loss import f1_loss
from utils.re_loss import re_loss
from utils.pre_loss import pre_loss

In [2]:
data_in = torch.tensor([[0.2,0.8,0,0],[0.1,0.8,0.1,0],[0.2,0.2,0.2,0.4],[0.2,0,0.7,0.1],[1.0,0,0,0],[0.2,0,0.7,0.1],[0.2,0.8,0,0]])
data_in.requires_grad = True

In [3]:
data_true = torch.tensor([0,1,3,2,2,2,3])

In [4]:
# class_weight = compute_class_weight('balanced', [0, 1, 2, 3], data_true.numpy())
# criterion = nn.CrossEntropyLoss(weight=torch.tensor(class_weight).float()).cuda()

In [5]:
data_in2 = data_in.clone()
data_in3 = data_in.clone()
data_in4 = data_in.clone()

In [6]:
data_in = data_in.cuda()
data_in2 = data_in2.cuda()
data_in3 = data_in3.cuda()
data_in4 = data_in4.cuda()
data_true = data_true.cuda()

In [7]:
p_f1 = []
p_re = []
p_pre = []
g_f1 = []
g_re = []
g_pre = []
epoch = 1000
l_rate = 0.01

for i in range(epoch):
    f1 = f1_loss(data_in2,data_true,n_class=4)
    data_in2.retain_grad()
    f1.backward()
    g_f1.append(data_in2.grad[0][0].cpu().detach().numpy().tolist())
    data_in2 = data_in2 - data_in2.grad*l_rate
    p_f1.append(data_in2[0][0].cpu().detach().numpy().tolist())
for i in range(epoch):
    re = re_loss(data_in3,data_true,n_class=4)
    data_in3.retain_grad()
    re.backward()
    g_re.append(data_in3.grad[0][0].cpu().detach().numpy().tolist())
    data_in3 = data_in3 - data_in3.grad*l_rate
    p_re.append(data_in3[0][0].cpu().detach().numpy().tolist())
for i in range(epoch):
    pre = pre_loss(data_in4,data_true,n_class=4)
    data_in4.retain_grad()
    pre.backward()
    g_pre.append(data_in4.grad[0][0].cpu().detach().numpy().tolist())
    data_in4 = data_in4 - data_in4.grad*l_rate
    p_pre.append(data_in4[0][0].cpu().detach().numpy().tolist())

[-0.0321233831346035]


NameError: name 'jj' is not defined

In [ ]:
p2_f1 = [(1-t)**2 for t in p_f1]
p2_re = [(1-t)**2 for t in p_re]
p2_pre = [(1-t)**2 for t in p_pre]

In [ ]:
plt.plot(np.arange(epoch),p2_f1)
plt.plot(np.arange(epoch),p2_re)
plt.plot(np.arange(epoch),p2_pre)
plt.legend(('f1', 're', 'pre'))
plt.title('A1-mse to epoch')
plt.xlabel('epoch')
plt.ylabel('A1-mse')
plt.show()

In [ ]:
plt.plot(np.arange(epoch),g_f1)
plt.plot(np.arange(epoch),g_re)
plt.plot(np.arange(epoch),g_pre)
plt.legend(('f1', 're', 'pre'))
plt.title('A1-grad to epoch')
plt.xlabel('epoch')
plt.ylabel('A1-grad')
plt.show()